# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
# List databases
databases = mongo.list_database_names()
print("Databases:", databases)

# Confirm the database is created
if 'uk_food' in databases:
    print("Database 'uk_food' exists.")

# Access the 'uk_food' database
db = mongo['uk_food']

# List collections in the database
collections = db.list_collection_names()
print("Collections in 'uk_food':", collections)

# Confirm the establishments collection is created
if 'establishments' in collections:
    print("Collection 'establishments' exists.")

# Find and display one document in the establishments collection
establishment = db.establishments.find_one()
pprint(establishment)


Databases: ['admin', 'config', 'local', 'uk_food']
Database 'uk_food' exists.
Collections in 'uk_food': ['establishments']
Collection 'establishments' exists.
{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'The Coastguard Inn',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 1034540,
 'LocalAuthorityBusinessID': 'PI/000078691',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-08-17T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('66fce98a995666ff459722d0'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'hre

In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [6]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": "",
    "AddressLine1": "Penang Flavours",
    "AddressLine2": "146A Plumstead Rd",
    "AddressLine3": "London",
    "AddressLine4": "",
    "PostCode": "SE18 7DY",
    "Phone": "",
    "LocalAuthorityCode": "511",
    "LocalAuthorityName": "Greenwich",
    "LocalAuthorityWebSite": "http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress": "health@royalgreenwich.gov.uk",
    "scores": {
        "Hygiene": "",
        "Structural": "",
        "ConfidenceInManagement": ""
    },
    "SchemeType": "FHRS",
    "geocode": {
        "longitude": "0.08384000",
        "latitude": "51.49014200"
    },
    "RightToReply": "",
    "Distance": 4623.9723280747176,
    "NewRatingPending": True
}

In [7]:
# Insert the new restaurant into the collection
insert_result = establishments.insert_one(new_restaurant)
print("New restaurant added with ID:", insert_result.inserted_id)

New restaurant added with ID: 66fcebfb39eb5f56ae9917b1


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [8]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
business_type = establishments.find_one(
    {"BusinessType": "Restaurant/Cafe/Canteen"},
    {"BusinessTypeID": 1, "BusinessType": 1}
)

# Print the result
pprint(business_type)
business_type_id = business_type.get('BusinessTypeID')

{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('66fce98a995666ff459722d1')}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [9]:
# Update the new restaurant with the correct BusinessTypeID
if business_type_id:
    establishments.update_one(
        {"_id": insert_result.inserted_id},
        {"$set": {"BusinessTypeID": business_type_id}}
    )
    print("Updated restaurant with BusinessTypeID:", business_type_id)

Updated restaurant with BusinessTypeID: 1


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [10]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print("Number of establishments in Dover:", dover_count)

# Remove any establishments within the Dover Local Authority
establishments.delete_many({"LocalAuthorityName": "Dover"})

# Check the number of documents to ensure they were deleted
new_dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print("Number of establishments in Dover after deletion:", new_dover_count)

Number of establishments in Dover: 994
Number of establishments in Dover after deletion: 0


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [11]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many(
    {},
    [
        {
            "$set": {
                "geocode.longitude": {"$toDouble": "$geocode.longitude"},
                "geocode.latitude": {"$toDouble": "$geocode.latitude"}
            }
        }
    ]
)
print("Converted latitude and longitude to decimal numbers.")

Converted latitude and longitude to decimal numbers.


Use `update_many` to convert `RatingValue` to integer numbers.

In [12]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [14]:
# Change the data type from String to Integer for RatingValue
establishments.update_many(
    {"RatingValue": {"$type": "string"}},
    [
        {
            "$set": {
                "RatingValue": {"$toInt": "$RatingValue"}
            }
        }
    ]
)
print("Converted RatingValue to integers where applicable.")
establishments.update_many(
    {},
    [
        {
            "$set": {
                "geocode.longitude": {"$toDouble": "$geocode.longitude"},
                "geocode.latitude": {"$toDouble": "$geocode.latitude"}
            }
        }
    ]
)
print("Converted latitude and longitude to decimal numbers.")

Converted RatingValue to integers where applicable.
Converted latitude and longitude to decimal numbers.


In [15]:
# Check that the coordinates and rating value are now numbers
sample_documents = establishments.find().limit(5)  # Retrieve 5 documents for checking
for doc in sample_documents:
    print("RatingValue:", doc.get("RatingValue"), "Type:", type(doc.get("RatingValue")))
    print("Latitude:", doc["geocode"].get("latitude"), "Type:", type(doc["geocode"].get("latitude")))
    print("Longitude:", doc["geocode"].get("longitude"), "Type:", type(doc["geocode"].get("longitude")))
    print("-" * 30)  # Separator for readability


RatingValue: 5 Type: <class 'int'>
Latitude: 51.083812 Type: <class 'float'>
Longitude: 1.195625 Type: <class 'float'>
------------------------------
RatingValue: 4 Type: <class 'int'>
Latitude: 51.086058 Type: <class 'float'>
Longitude: 1.196408 Type: <class 'float'>
------------------------------
RatingValue: 5 Type: <class 'int'>
Latitude: 51.085797 Type: <class 'float'>
Longitude: 1.194762 Type: <class 'float'>
------------------------------
RatingValue: 5 Type: <class 'int'>
Latitude: 51.08084 Type: <class 'float'>
Longitude: 1.188537 Type: <class 'float'>
------------------------------
RatingValue: 5 Type: <class 'int'>
Latitude: 51.08084 Type: <class 'float'>
Longitude: 1.188537 Type: <class 'float'>
------------------------------
